<a href="https://colab.research.google.com/github/Bandi120424/KDT-AI/blob/Month-3/Spark%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName('Titanic Binary Classification Example') \
    .getOrCreate()

In [ ]:
spark

### 타이타닉 생존 예측 모델 만들기

In [ ]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/titanic.csv

--2022-12-19 05:00:59--  https://s3-geospatial.s3-us-west-2.amazonaws.com/titanic.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.92.163.50, 52.218.242.249, 52.92.210.34, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.92.163.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 61197 (60K) [text/csv]
Saving to: ‘titanic.csv.3’

titanic.csv.3       100%[===================>]  59.76K  --.-KB/s    in 0.08s   

2022-12-19 05:01:00 (710 KB/s) - ‘titanic.csv.3’ saved [61197/61197]



In [ ]:
data = spark.read.csv('./titanic.csv', header = True, inferSchema = True)
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
data.show(5)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|Gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

In [ ]:
data.select(['*']).describe().show()

+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|summary|      PassengerId|           Survived|            Pclass|                Name|Gender|               Age|             SibSp|              Parch|            Ticket|             Fare|Cabin|Embarked|
+-------+-----------------+-------------------+------------------+--------------------+------+------------------+------------------+-------------------+------------------+-----------------+-----+--------+
|  count|              891|                891|               891|                 891|   891|               714|               891|                891|               891|              891|  204|     889|
|   mean|            446.0| 0.3838383838383838| 2.308641975308642|                null|  null| 29.69911764705882|0.5230078563411896|0.38159371492704824|260318.54916792738| 32.20420

In [ ]:
data.select('Ticket').distinct().show()

+----------------+
|          Ticket|
+----------------+
|          367230|
|       P/PP 3381|
|          244270|
|          363291|
|SOTON/OQ 3101317|
|           31418|
|           26360|
|            2700|
|           14313|
|          345763|
|       A/5 21172|
|          364500|
|           19877|
|          350029|
|          113800|
|      A/4. 39886|
|       C.A. 2673|
|          113807|
|        PC 17604|
|       C.A. 2315|
+----------------+
only showing top 20 rows



In [ ]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Gender',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [ ]:
final_data = data.select(['Survived',
 'Pclass',
 'Gender',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
])

In [ ]:
final_data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)



Age는 평균값으로 채움 

In [ ]:
from pyspark.ml.feature import Imputer

imputer = Imputer(strategy = 'mean', inputCols = ['Age'], outputCols = ['AgeImputed'])
imputer_model = imputer.fit(final_data)
final_data = imputer_model.transform(final_data)

In [ ]:
final_data.select('Age','AgeImputed').show()

+----+-----------------+
| Age|       AgeImputed|
+----+-----------------+
|22.0|             22.0|
|38.0|             38.0|
|26.0|             26.0|
|35.0|             35.0|
|35.0|             35.0|
|null|29.69911764705882|
|54.0|             54.0|
| 2.0|              2.0|
|27.0|             27.0|
|14.0|             14.0|
| 4.0|              4.0|
|58.0|             58.0|
|20.0|             20.0|
|39.0|             39.0|
|14.0|             14.0|
|55.0|             55.0|
| 2.0|              2.0|
|null|29.69911764705882|
|31.0|             31.0|
|null|29.69911764705882|
+----+-----------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.feature import StringIndexer

gender_indexer = StringIndexer(inputCol = 'Gender', outputCol = 'GenderIndexed')
gender_indexer_model = gender_indexer.fit(final_data)
final_data = gender_indexer_model.transform(final_data)

In [ ]:
final_data.select('Gender',  'GenderIndexed').show(5)

+------+-------------+
|Gender|GenderIndexed|
+------+-------------+
|  male|          0.0|
|female|          1.0|
|female|          1.0|
|female|          1.0|
|  male|          0.0|
+------+-------------+
only showing top 5 rows



### 피쳐 벡터 만들기

In [ ]:
final_data.columns

['Survived',
 'Pclass',
 'Gender',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'AgeImputed',
 'GenderIndexed']

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = [
 'Pclass',
 'SibSp',
 'Parch',
 'Fare',
 'AgeImputed',
 'GenderIndexed'], outputCol = 'featureVec')
data_vec = assembler.transform(final_data)

In [ ]:
data_vec.show(3)

+--------+------+------+----+-----+-----+-------+----------+-------------+--------------------+
|Survived|Pclass|Gender| Age|SibSp|Parch|   Fare|AgeImputed|GenderIndexed|          featureVec|
+--------+------+------+----+-----+-----+-------+----------+-------------+--------------------+
|       0|     3|  male|22.0|    1|    0|   7.25|      22.0|          0.0|[3.0,1.0,0.0,7.25...|
|       1|     1|female|38.0|    1|    0|71.2833|      38.0|          1.0|[1.0,1.0,0.0,71.2...|
|       1|     3|female|26.0|    0|    0|  7.925|      26.0|          1.0|[3.0,0.0,0.0,7.92...|
+--------+------+------+----+-----+-----+-------+----------+-------------+--------------------+
only showing top 3 rows



In [ ]:
data_vec.select('featureVec').show()

+--------------------+
|          featureVec|
+--------------------+
|[3.0,1.0,0.0,7.25...|
|[1.0,1.0,0.0,71.2...|
|[3.0,0.0,0.0,7.92...|
|[1.0,1.0,0.0,53.1...|
|[3.0,0.0,0.0,8.05...|
|[3.0,0.0,0.0,8.45...|
|[1.0,0.0,0.0,51.8...|
|[3.0,3.0,1.0,21.0...|
|[3.0,0.0,2.0,11.1...|
|[2.0,1.0,0.0,30.0...|
|[3.0,1.0,1.0,16.7...|
|[1.0,0.0,0.0,26.5...|
|[3.0,0.0,0.0,8.05...|
|[3.0,1.0,5.0,31.2...|
|[3.0,0.0,0.0,7.85...|
|[2.0,0.0,0.0,16.0...|
|[3.0,4.0,1.0,29.1...|
|[2.0,0.0,0.0,13.0...|
|[3.0,1.0,0.0,18.0...|
|[3.0,0.0,0.0,7.22...|
+--------------------+
only showing top 20 rows



In [ ]:
data_vec.iloc[0].collect() #sql.Dataframe 에는 iloc 없음!

AttributeError: ignored

In [ ]:
data_vec.collect()[0]['featureVec']

DenseVector([3.0, 1.0, 0.0, 7.25, 22.0, 0.0])

train/test 데이터 분리

In [ ]:
train, test = data_vec.randomSplit([0.7, 0.3])

In [ ]:
from pyspark.ml.classification import LogisticRegression

algo = LogisticRegression(featuresCol = 'featureVec', labelCol = 'Survived')
model = algo.fit(train)

모델 성능 측정

In [ ]:
predictions = model.transform(test)
predictions.select(['Survived', 'prediction', 'probability']).show() #threshold = 0.5

+--------+----------+--------------------+
|Survived|prediction|         probability|
+--------+----------+--------------------+
|       0|       1.0|[0.04646237626102...|
|       0|       0.0|[0.54009259355509...|
|       0|       0.0|[0.53391259457221...|
|       0|       0.0|[0.53351186636077...|
|       0|       0.0|[0.50841803575443...|
|       0|       0.0|[0.50577190630382...|
|       0|       1.0|[0.42170502828829...|
|       0|       1.0|[0.31822565249080...|
|       0|       1.0|[0.21170906264156...|
|       0|       1.0|[0.26115408548333...|
|       0|       0.0|[0.52220844803869...|
|       0|       0.0|[0.62989532840920...|
|       0|       0.0|[0.60315855452994...|
|       0|       0.0|[0.60261340465013...|
|       0|       0.0|[0.74809514642970...|
|       0|       0.0|[0.79038132352996...|
|       0|       0.0|[0.70230920677319...|
|       0|       0.0|[0.82039039319159...|
|       0|       0.0|[0.81946880875879...|
|       0|       0.0|[0.83117878508299...|
+--------+-

threshold 값을 변경해보자

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol = 'Survived', metricName = 'areaUnderROC')
evaluator.evaluate(predictions)

0.8553156748911473